In this notebook, different statistics of the detected ramp planes will be analyzed and visualized.<br>
Also a score will be calculated to determine how well the algorithm works.<br>
This will be done for multiple rosbags to get an average score.

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
# Load offline ramp detection algorithm
from lidar_ramp_offline import VisualDetection
# Load methods to extract data from rosbags
from getData import unpack_bag, synchronize_topics
# Load custom plots
from plot_library_lidar import PlotLib

pio.templates.default = "plotly_dark"
# %load_ext blackcellmagic

#### **The following values need to be adjusted**

In [ ]:
# Rosbags path
bag_directory = '/home/user/rosbags/final/slam/'

# ROS topics
odom_topic = '/odom'
lidar_topic = '/velodyne_points'

# Names of rosbags to evaluate
bag_names = [
    "u_c2s_half_odom_hdl.bag",
    'u_c2s_half_odom_stereo_hdl.bag',
    "u_c2s_hdl.bag",
    "u_c2s_stop_hdl.bag",
    "u_d2e_hdl.bag",
    "u_s2c_half_odom_hdl.bag"
]

# Ground truth ramp region for each bag
xy_range = [
    # u_c2s_half_odom
    [[20.3, 33], [-0.9, 2.8]],
    # u_c2s_half_odom_stereo (quite a bit rotated)
    [[23.8, 36], [-3.3, 0.5]],
    # u_c2s
    [[45, 58], [-1.9, 1.8]],
    # u_c2s_stop
    [[38, 51], [-1.5, 2.2]],
    # u_d2e
    [[32.5, 44], [2, 5.5]],
    # u_s2c_half (hard because ramp is not straight)
    [[42, 56], [-2.2, 2]],
]

### Evaluate output/performance of detection algorithm

In [ ]:
class GetScore:
    def __init__(self, bag_name, x_range, y_range):
        self.bag_name = bag_name
        self.x_range = x_range
        self.y_range = y_range

    def extract_data(self, bag_path):
        """Get odometer and lidar data from rosbag"""
        # Odometer from hdl_slam
        pose_async, t_pose = unpack_bag(bag_path, odom_topic, 'hdl_odom')
        # Velodyne point cloud
        lidar_async, t_lidar = unpack_bag(bag_path, lidar_topic)

        # Synchronize both topics (odom usually is behind lidar)
        pose, lidar = synchronize_topics(pose_async, t_pose, lidar_async, t_lidar)
        return lidar, pose

    def run_the_algorithm(self, lidar, pose):
        """Perform ramp detection on data"""
        # Create instance of class (using standard parameters):
        vd = VisualDetection(x_range[0])
        # Lists to fill, will contain entry for each plane
        planes = []
        ramp_stats = []
        true_dist = []
        all_points = []
        for i in range(len(lidar)):
            plane_points, data, pc_whole = vd.spin(lidar[i], pose[i])
            planes.append(plane_points)
            ramp_stats.append(data)
            all_points.append(pc_whole)
            # True distance to start of ramp
            true_dist.append(self.x_range[0] - pose[i].position.x)
        true_dist = np.array(true_dist)
        return planes, ramp_stats, true_dist, all_points

    def convert_to_df(self, planes, ramp_stats):
        """Add pointcloud to df each time a ramp has been detected"""
        # Remove empty lists (when no ramp has been detected)
        # because information such as angle, width of ramp etc
        # are only stored when a ramp is detected, 0 otherwise
        ramp_arrays = [x for x in planes if not isinstance(x, list)]
        ramp_stats = [x for x in ramp_stats if x != []]
        # Get indices where ramp has been detected
        ramp_indices = [i for i, v in enumerate(planes) if not isinstance(v, list)]

        # Convert list to dictionary
        dic = []
        for i, arr in enumerate(ramp_arrays):
            for j, point in enumerate(arr):
                dic.append(
                    {
                        "sampleIdx": i,
                        "pointIdx": j,
                        "x": point[0],
                        "y": point[1],
                        "z": point[2],
                    }
                )
        # And finally to pandas data frame
        df = pd.DataFrame(dic)
        # Reorder columns
        df = df[["sampleIdx", "pointIdx", "x", "y", "z"]]
        return df, ramp_stats, ramp_indices

    def ground_truth_check(self, df, ramp_stats, only_before_ramp=True):
        """Check if ?"""
        # Check if a point lies within ramp region
        lies_inside = []
        for i, x in enumerate(df["x"]):
            if self.x_range[0] < x < self.x_range[1]:
                if self.y_range[0] < df["y"][i] < self.y_range[1]:
                    # True if x and y coordinate inside region
                    lies_inside.append(True)
                else:
                    lies_inside.append(False)
            else:
                lies_inside.append(False)
        # Add column (bool: if point lies in region) to data frame
        df["inlier"] = lies_inside

        # Calculate how many points of each sample lie in ramp region
        true_inliers = []
        samples_num = df.sampleIdx.max() + 1
        for i in range(samples_num):
            # Bool list of inliers and outliers of sample
            bool_lst = df[df["sampleIdx"] == i]["inlier"]
            # Percentage of inliers of sample
            true_inliers.append(sum(bool_lst) / float(len(bool_lst)))

        # New dataframe with stats for each frame
        # Structure reminder of ramp_stats: [angle, width, dist, true_dist]
        dic = []
        for i in range(samples_num):
            dic.append(
                {
                    "sampleIdx": i,
                    "TrueInliers": true_inliers[i],
                    "Angle": ramp_stats[i][0],
                    "Width": ramp_stats[i][1],
                    "Dist": ramp_stats[i][2],
                    "TrueDist": ramp_stats[i][3],
                }
            )
        # Convert dictionary to dataframe
        df_stats = pd.DataFrame(dic)
        # Reorder columns
        df_stats = df_stats[
            ["sampleIdx", "TrueInliers", "Angle", "Width", "Dist", "TrueDist"]
        ]
        # Remove all estimations from when after the ramp has been entered
        if only_before_ramp:
            df_stats = df_stats[df_stats["TrueDist"] > 0]
        return df_stats

    def calc_score(self, df_stats, true_dist, min_dist, max_dist):
        """Calculates some scores to determine performance"""
        # How many sample were recorded in given range
        expected_detections = len(
            true_dist[(min_dist < true_dist) & (true_dist < max_dist)]
        )
        # How many samples were identified as ramp in given range
        actual_detections = len(
            df_stats["TrueDist"][
                (min_dist < df_stats["TrueDist"]) & (df_stats["TrueDist"] < max_dist)
            ]
        )
        # Calculate ratio
        sensitivity = float(actual_detections) / expected_detections
        print(self.bag_name)
        print("Out of {} recorded samples {} were detected in the range of {} to {}m before the ramp.".format(
                expected_detections, actual_detections, min_dist, max_dist))
        print("Resulting in a score of {:.2f}%\n".format(sensitivity * 100))

        # Inlier score
        f = df_stats["TrueInliers"][
            (min_dist < df_stats["TrueDist"]) & (df_stats["TrueDist"] < max_dist)]
        score2 = sum(f) / len(f)
        print("Of the {} detected planes {:.2f}% inlier points were actually inside the ramp region.\n".format(
                actual_detections, score2 * 100))

        # False positives?
        f = df_stats["TrueInliers"][
            (min_dist < df_stats["TrueDist"]) & (df_stats["TrueDist"] < max_dist)
        ]
        thresh = 0.5
        true_detections = sum(f > thresh)
        score3 = float(true_detections) / len(f)
        print("Of the {} detected planes {} had at least {}% of points inside the ramp region.".format(
            actual_detections, true_detections, int(thresh * 100)))
        print("Resulting in a score of {:.2f}%\n\n".format(score3 * 100))

        scores = [sensitivity, score2, score3]
        return scores
    
    def vis_score(self, df_stats, true_dist, only_before_ramp=True):
        expected_detections = []
        actual_detections = []
        # Split dataset in 1m intervals
        for i in range(15, -5, -1):
            # Filter distance to ramp (from odom data) in 1m intervals
            expected_detections.append(
                len(true_dist[(i < true_dist) & (true_dist < i + 1)]))
            # Get corresponding distance estimations
            actual_detections.append(
                len(
                    df_stats["TrueDist"][
                        (i < df_stats["TrueDist"]) & (df_stats["TrueDist"] < i + 1)
                    ]
                )
            )
        # Create pandas data frame
        df_eval = pd.DataFrame()
        # Distance to ramp e.g. a value of 15 means interval from 16m to 15m
        df_eval["distToRamp"] = range(15, -5, -1)
        df_eval["expectedDetections"] = expected_detections
        df_eval["actualDetections"] = actual_detections
        # Remove all estimations from when after the ramp has been entered
        if only_before_ramp:
            df_eval = df_eval[df_eval["distToRamp"] > 0]
        return df_eval
    
    def plotting(self, df, df_stats, df_eval, x_range, y_range, all_points, ramp_indices):
        plt = PlotLib(df, df_stats, df_eval, x_range, y_range, all_points, ramp_indices)
        # plt.bar_plot().show()
        plt.distance_estimation().show()
        plt.angle_estimation().show()
        plt.width_estimation().show()

    def boss_method(self, min_dist=1, max_dist=15, show_plot=True):
        """Run all methods above """
        # Load rosbag
        bag_path = bag_directory + self.bag_name
        lidar, pose = self.extract_data(bag_path)
        planes, ramp_stats, true_dist, all_points = self.run_the_algorithm(lidar, pose)
        df, ramp_stats, ramp_indices = self.convert_to_df(planes, ramp_stats)
        df_stats = self.ground_truth_check(df, ramp_stats)
        df_eval = self.vis_score(df_stats, true_dist)
        if show_plot:
            self.plotting(df, df_stats, df_eval, x_range, y_range, all_points, ramp_indices)
        #     self.some_plot(true_dist, df_stats)
        #     self.more_plots(df_stats)
        scores = self.calc_score(df_stats, true_dist, min_dist, max_dist)
        return scores

### Run algorithm and evaluation for each rosbag

In [ ]:
scores = []
for i, bag_name in enumerate(bag_names):
    # Extract ramp region data
    x_range, y_range = xy_range[i]
    # Create instance of class
    gs = GetScore(bag_name, x_range, y_range)
    # Run algorithm + evaluation
    score = gs.boss_method()
    scores.append(score)